In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
import sparknlp

from sparknlp.base import EmbeddingsFinisher
from sparknlp.annotator import DocumentAssembler, Tokenizer, XlmRoBertaEmbeddings
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

import pandas as pd

from models.bloom_filter import BloomFilter

In [2]:
spark = SparkSession.builder \
    .appName("WikimediaStreamProcessor") \
    .config("spark.driver.memory", "16G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.5.1") \
    .getOrCreate()


24/11/16 17:46:33 WARN Utils: Your hostname, ucuvm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/11/16 17:46:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andrii/.ivy2/cache
The jars for the packages stored in: /home/andrii/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-715c4040-93b2-4e8f-9d75-fa5ffe4d9e41;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in centra

In [3]:
train_df = spark.read.csv(
    "./data/train/train_dataset.csv",
    header=True,
    inferSchema=True,
    multiLine=True,
    escape="\"",
    quote="\"",
    sep=",",
)

In [4]:
bots_list = train_df.filter(F.col("bot")).select('user').distinct().rdd.map(lambda r: r[0]).collect()

filter = BloomFilter(capacity=len(bots_list), false_positive_rate=0.1)
for bot in bots_list:
    filter.add(bot)

In [5]:
filter.save('./data/filter_train_small')

In [6]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = XlmRoBertaEmbeddings.pretrained() \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings") \
    .setCaseSensitive(True)

embeddingsFinisher = EmbeddingsFinisher() \
    .setInputCols(["embeddings"]) \
    .setOutputCols("finished_embeddings") \
    .setOutputAsVector(True) \
    .setCleanAnnotations(False)

pipeline = Pipeline() \
    .setStages([
      documentAssembler,
      tokenizer,
      embeddings,
      embeddingsFinisher
    ])

xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
[ | ]

24/11/16 17:47:10 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/11/16 17:47:10 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


xlm_roberta_base download started this may take some time.
Approximate size to download 619.5 MB
Download done! Loading the resource.
[ / ]

[ | ]#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGILL (0x4) at pc=0x000076a4372bc260, pid=17854, tid=17949
#
# JRE version: OpenJDK Runtime Environment Homebrew (11.0.25) (build 11.0.25+0)
# Java VM: OpenJDK 64-Bit Server VM Homebrew (11.0.25+0, mixed mode, tiered, compressed oops, g1 gc, linux-amd64)
# Problematic frame:
# C  [libtensorflow_framework.so.2+0x16bc260]  nsync::nsync_mu_init(nsync::nsync_mu_s_*)+0x0
#
# Core dump will be written. Default location: Core dumps may be processed with "/usr/share/apport/apport -p%p -s%s -c%c -d%d -P%P -u%u -g%g -- %E" (or dumping to /media/andrii/partition_2/MMDS/final_project/core.17854)
#
# An error report file with more information is saved as:
# /media/andrii/partition_2/MMDS/final_project/hs_err_pid17854.log
#
# If you would like to submit a bug report, please visit:
#   https://github.com/Homebrew/homebrew-core/issues
# The crash happened outside the Java Virtual Machine in native code.
# See problematic f

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 48636)
Traceback (most recent call last):
  File "/home/linuxbrew/.linuxbrew/opt/python@3.13/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/linuxbrew/.linuxbrew/opt/python@3.13/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/linuxbrew/.linuxbrew/opt/python@3.13/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/linuxbrew/.linuxbrew/opt/python@3.13/lib/python3.13/socketserver.py", line 761, in __init__
    self.handle()
    ~~~~~~~~~~~^^
  File "/media/andrii/partition_2/MMDS/venv/

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
        "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving


[OK!]


Py4JError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/media/andrii/partition_2/MMDS/venv/lib/python3.13/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
        "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
train_df_wtext = train_df.withColumn(
    "text",
    F.concat(
        F.lit("title: "), F.col("title"),
        F.lit("; comment: "),
        F.when(F.col("comment").isNotNull(), F.col("comment")).otherwise(F.lit("NULL"))
    )
)

ConnectionRefusedError: [Errno 111] Connection refused